<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/12_chat_with_history.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 12. Chat with History

## Overview  
This exercise demonstrates the implementation of a question-answering system that utilizes user conversation history. It constructs a chain to generate appropriate responses to user questions based on previous conversation records. Additionally, it allows for more precise question-answering tasks by providing specific context.

## Purpose of the Exercise
The purpose of this exercise is to enhance the ability to manage questions and responses using conversation history. By the end of this tutorial, users will understand how to create a system that uses past interactions to inform current responses, thereby improving the accuracy and relevance of the answers provided.


In [ ]:
! pip3 install -qU langchain-upstage  python-dotenv python-dotenv

In [2]:
# @title set API key
from pprint import pprint
import os

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

assert (
    "UPSTAGE_API_KEY" in os.environ
), "Please set the UPSTAGE_API_KEY environment variable"

In [3]:
from langchain_upstage import ChatUpstage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


llm = ChatUpstage(model="solar-pro")

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an assistant for question-answering tasks. "),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

chain = qa_prompt | llm | StrOutputParser()

In [5]:
from langchain_core.messages import HumanMessage

question = "How about Korea?"
ai_msg_1 = chain.invoke({"input": question, "chat_history": []})
print(ai_msg_1)

Could you please provide more context or specify what information you need about Korea? Are you interested in its geography, history, culture, or something else?


In [6]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []

question = "Where is the capital of France?"
ai_msg_1 = chain.invoke({"input": question, "chat_history": chat_history})
print(ai_msg_1)
chat_history.extend([HumanMessage(question), AIMessage(ai_msg_1)])


second_question = "How about Korea?"
ai_msg_2 = chain.invoke({"input": second_question, "chat_history": chat_history})
chat_history.extend([HumanMessage(second_question), AIMessage(ai_msg_2)])

print(ai_msg_2)

The capital of France is Paris.
The capital of South Korea is Seoul, while the capital of North Korea is Pyongyang.


In [7]:
llm = ChatUpstage(model="solar-pro")

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

chain = qa_prompt | llm | StrOutputParser()

In [8]:
context = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters,
    demonstrating superior performance in various natural language processing (NLP) tasks.
    Inspired by recent efforts to efficiently up-scale LLMs,
    we present a method for scaling LLMs called depth up-scaling (DUS),
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts,
    DUS does not require complex changes to train and inference efficiently.
    We show experimentally that DUS is simple yet effective
    in scaling up high-performance LLMs from small ones.
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct,
    a variant fine-tuned for instruction-following capabilities,
    surpassing Mixtral-8x7B-Instruct.
    SOLAR 10.7B is publicly available under the Apache 2.0 license,
    promoting broad access and application in the LLM field.
"""

In [9]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []

question = "What is DUS?"
ai_msg_1 = chain.invoke(
    {"input": question, "chat_history": chat_history, "context": context}
)
chat_history += [HumanMessage(question), AIMessage(ai_msg_1)]
print("A1", ai_msg_1)

second_question = "What's the benefit?"
ai_msg_2 = chain.invoke(
    {"input": second_question, "chat_history": chat_history, "context": context}
)
chat_history += [HumanMessage(second_question), AIMessage(ai_msg_2)]

print("A2", ai_msg_2)

A1 DUS stands for Depth Up-Scaling, a method for scaling large language models (LLMs) introduced in the context of SOLAR 10.7B. It involves depthwise scaling and continued pretraining, and is simpler yet effective compared to other LLM up-scaling methods that use mixture-of-experts.
A2 The benefit of DUS is that it allows for efficient scaling up of high-performance LLMs from smaller ones without requiring complex changes for efficient training and inference, as some other methods do.


In [11]:
for chat in chat_history:
    print(chat)

content='What is DUS?' additional_kwargs={} response_metadata={}
content='DUS stands for Depth Up-Scaling, a method for scaling large language models (LLMs) introduced in the context of SOLAR 10.7B. It involves depthwise scaling and continued pretraining, and is simpler yet effective compared to other LLM up-scaling methods that use mixture-of-experts.' additional_kwargs={} response_metadata={}
content="What's the benefit?" additional_kwargs={} response_metadata={}
content='The benefit of DUS is that it allows for efficient scaling up of high-performance LLMs from smaller ones without requiring complex changes for efficient training and inference, as some other methods do.' additional_kwargs={} response_metadata={}
